In [ ]:
import os
from openai import OpenAI


def main():
    # Load your API key from environment
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        raise ValueError("Please set the OPENAI_API_KEY environment variable")

    client = OpenAI(api_key=api_key)

    # The query
    user_query = "Tell me upcoming game for mexico national team of men"

    # Call the Responses API with web search tool
    response = client.responses.create(
        model="gpt-4o",
        input=user_query,
        instructions=prompt,
        tools=[
            {
                "type": "web_search_preview",
                "user_location": {
                    "type": "approximate",
                    "country": "MX"
                },
                "search_context_size": "medium"
            }
        ]
    )

    # Extract output text
    output = response.output_text
    print("Assistant Response:\n", output)


if __name__ == "__main__":
    main()


In [ ]:
import os
from openai import OpenAI

prompt = """
You are an AI assistant specialized in football (soccer). When answering a user’s query:

1. **Always use the web search tool** to fetch the most recent football data (fixtures, results, standings, stats, transfers, etc.).
2. **Do not include disclaimers, reference links, or source mentions** in your response.
3. **Return only the factual, up-to-date results** in a clean, structured, and concise format (tables for standings, bullet points for matches, short lists for stats).
4. If multiple possible answers exist (e.g., multiple leagues or teams), ask the user for clarification before responding.
5. If no current data is found, simply reply: **“No current data available.”**

**User Example Queries:**

* “Upcoming La Liga matches this weekend”
* “Champions League results today”
* “Serie A top scorers right now”
* “Premier League standings”

**Expected Output Style (example):**
Upcoming La Liga Matches:

* Real Madrid vs. Valencia — Sept 21, 20:00 CET
* Barcelona vs. Sevilla — Sept 22, 18:30 CET
* Atlético Madrid vs. Villarreal — Sept 23, 21:00 CET
"""

def chat_loop():
    # Load API key
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        raise ValueError("Set the OPENAI_API_KEY environment variable")

    client = OpenAI(api_key=api_key)

    # Conversation history (list of messages)
    history = []
    # Optionally include a system prompt
    history.append({"role": "system", "content": prompt})

    print("Chat with assistant (type 'exit' to quit)")

    while True:
        user_input = input("\nYou: ")
        if user_input.strip().lower() in ["exit", "quit"]:
            print("Goodbye!")
            break

        # Append user message to history
        history.append({"role": "user", "content": user_input})

        # Call the Responses API, with web search tool enabled
        resp = client.responses.create(
            model="gpt-4o",
            input=history,
            tools=[
                {
                    "type": "web_search_preview",
                    "user_location": {
                        "type": "approximate",
                        "country": "MX"
                    },
                    "search_context_size": "medium"
                }
            ]
        )

        assistant_message = resp.output_text.strip()
        print("\nAssistant:", assistant_message)

        # Append assistant response back to history
        history.append({"role": "assistant", "content": assistant_message})
